In [2]:
import spacy
from spacy import displacy
from os import listdir

nlp = spacy.load('en_core_web_sm')

In [78]:
PERSONS = ['I', 'ME', 'MY', 'MINE', 'YOU', 'YOUR', 'YOURS', 'HE', 'SHE', 'HIS', 'HER', 'HIM', 'THEY', 'THEM', 'THEMSELVES', 'OUR', 'WE']
LOCATIONS = ['LOC', 'GPE', 'ORG', 'FAC']
ALL_ENTITIES = LOCATIONS + ['PERSON', 'HEALTH', 'MONEY', 'DATE', 'TIME']

**Non Disclosure Experiment Data**

In [79]:
# for filename in sorted(listdir('data/non_disclosure')):
#     if not filename.endswith('.txt'):
#         continue
        
#     with open('data/non_disclosure/' + filename) as f:
#         content = f.read()
#         doc = nlp(u''+content)
#         # print(content)
#         displacy.render(doc, style='ent', jupyter=True) 
#         # print('-'*50)

**Disclosure Experiment Data**

In [80]:
# for filename in sorted(listdir('data/disclosure')):
#     if not filename.endswith('.txt'):
#         continue
        
#     with open('data/disclosure/' + filename) as f:
#         content = f.read()
#         doc = nlp(u''+content)
#         # print(content)
#         displacy.render(doc, style='ent', jupyter=True) 
#         # print('-'*50)

### Manual Labeling of Data

In [81]:
from os import listdir
from string import punctuation
import nltk.data

In [82]:
def write_docs(dnd, data):
    with open(dnd + '.tsv', 'a') as f:
        f.write(data)
        print('Writing file done!')
        

In [83]:
def process_docs(path):
    with open(path, 'r') as f:
        non_disclosure = []
        disclosure = []

        text = f.read()
        
        sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        lines = sent_detector.tokenize(text.strip())
        
        for line in lines:
            # remove punctuation from each token. Except ', char
            tokens = line.split()
            print(tokens)
            table = str.maketrans('', '', '!"#$%&()*+-./:;<=>?@[\]^_`{|}~')
            tokens = [w.translate(table) for w in tokens]
            doc = ' '.join(tokens)

            doc = nlp(u''+doc)
            new_doc = ''

            entity = 0
            spacy_tokens = []
            
            for t in doc:
                spacy_tokens.append(t.text)
                
                if t.ent_type_ == '':
                    if t.text.upper() in PERSONS:
                        entity = entity+1
                        
                        index = spacy_tokens.index(t.text)
                        ## Replace with Entity Label
                        # spacy_tokens[index] = 'PERSON'
                    else:
                        pass
                elif t.ent_type_ in ALL_ENTITIES:
                    entity = entity+1
                    
                    index = spacy_tokens.index(t.text)
                    ## Replace with Entity Label
                    # spacy_tokens[index] = t.ent_type_.upper()                    
            
            new_doc = ' '.join(spacy_tokens)
            
            if entity > 1:
                if len(set([t.upper() for t in spacy_tokens]).intersection(set(PERSONS))) > 0:
                    # print('DISCLOSURE: ', new_doc)
                    # disclosure.append(new_doc)
                    write_docs('Manually_Labeled_Data/disclosure', '1\t' + new_doc + '\n')
                    continue
                else:
                    # print('NON-DISCLOSURE: ', new_doc)
                    # non_disclosure.append(new_doc)
                    write_docs('Manually_Labeled_Data/non-disclosure', '0\t' + new_doc + '\n')
                    continue            
            else:
                # print('NON-DISCLOSURE: ', new_doc)
                # non_disclosure.append(new_doc)
                write_docs('Manually_Labeled_Data/non-disclosure', '0\t' + new_doc + '\n')
                continue     
    
    print('\n')

In [87]:
directory = 'data/medical_posts'

# Make sure have new blank files
# with open('Manually_Labeled_Data/non-disclosure.tsv', 'w') as f:
#     f.write('')
# with open('Manually_Labeled_Data/disclosure.tsv', 'w') as f:
#     f.write('')  

i = 0
for filename in sorted(listdir(directory)):
    if not filename.endswith(".txt"):
        continue
        
#     if i >= 2:
#         break      
        
    print(filename)
    path = directory + '/' + filename    
    process_docs(path)
    i = i+1
    

In [144]:
# from string import punctuation

# with open('Temp/medical.txt', 'r') as f:
    
#     sentences = []
#     needed_sentences = []
    
#     non_disclosure = []
#     disclosure = []
    
#     lines = f.readlines()
#     for line in lines:
#         splitted_line = line.split('   ')
#         if len(splitted_line) >= 3:
#             sentence = splitted_line[2]
            
#             tokens = sentence.split()
#             # remove punctuation from each token
#             table = str.maketrans('', '', punctuation)
#             tokens = [w.translate(table) for w in tokens]
#             doc = ' '.join(tokens)

#             doc = nlp(u''+doc)
#             new_doc = ''
            
#             entity = 0            
#             for t in doc:
#                 if t.ent_type_ == '':
#                     if t.text.upper() in PERSONS:
#                         entity = entity+1
#                         new_doc = doc.text.replace(' '+t.text+' ', ' PERSON ')
#                         doc = nlp(u''+new_doc)
#                     else:
#                         pass
#                 elif t.ent_type_ in ALL_ENTITIES:
#                     entity = entity+1
#                     new_doc = doc.text.replace(t.text, t.ent_type_.upper())
#                     doc = nlp(u''+new_doc)
            
#             if entity != 0:
#                 sentence = new_doc.strip() + ' #OFENTFOUND ' + str(entity)
#                 number_of_ents = int(sentence.split('#OFENTFOUND')[1])
#                 if number_of_ents > 1:
#                     tokens = sentence.split('#OFENTFOUND')[0].split()
#                     if 'PERSON' in tokens:
#                         print('diclosure')
#                         disclosure.append(new_doc.strip() + ' #OFENTFOUND ' + str(entity))
#                         continue
#                     else:
#                         print('non-diclosure')
#                         non_disclosure.append(new_doc.strip() + ' #OFENTFOUND ' + str(entity))
#                         continue            
#                 else:
#                     print('non-diclosure')
#                     non_disclosure.append(new_doc.strip() + ' #OFENTFOUND ' + str(entity))
#                     continue                
#             else:
#                 print('non-diclosure')
#                 non_disclosure.append(new_doc.strip() + ' #OFENTFOUND ' + str(entity))
                
#             sentences.append(sentence)
#         else:
#             pass
        
# print('Disclosure Sentences:', len(disclosure))
# print('Non disclosure Sentences:', len(non_disclosure), '\n')                

**Finding a verb with a subject otherwise expand the context**
To Do: R&D

In [41]:
# # Because the syntactic relations form a tree, every word has exactly one head.
# from spacy.symbols import nsubj, VERB

# first_sentence = "How is your son?"
# second_sentence = "Got the flu."
# context = ""

# doc = nlp(u''+second_sentence)

# displacy.render(doc, style='dep', jupyter=True, options={'distance' : 200})

# verbs = set()
# for possible_subject in doc:
#     if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
#         verbs.add(possible_subject.head)
        
# if len(verbs) == 0:
#     context = first_sentence + " " + second_sentence
#     print(context)       

# doc = nlp(u''+context.lower())
# displacy.render(doc, style='dep', jupyter=True, options={'distance' : 200})

# context = []
# for token in doc:
#     if token.dep_ == 'nsubj' or token.dep_ == 'dobj' or token.dep_ == 'ROOT':
#         context.append(str(token))
    
# print(' '.join(context))    

**Creating numbered files from texts**

In [120]:
import csv
from random import shuffle
import random

with open('Manually_Labeled_Data/non-disclosure.tsv', encoding='utf-8') as csvfile:
    readCSV = csv.reader(csvfile, delimiter='\t')
    
    sentences = []
    i = 0
    for row in readCSV:
        review = row[1]
        sentences.append(review)
        i += 1         
        
# print(sentences, len(sentences))
sentences = random.sample(sentences, len(sentences))
# print(sentences)

j = 0
for sentence in sentences:
    if j >= 5000:
        break
    print(sentence)
    with open("data/nd/"+str(j).zfill(5)+"_line.txt", "w") as f:
        f.write(sentence.strip())
    j += 1

And I thought that everything is going well
Free breakfast was great dinner not so much
Was given 2 towels and washclothes for a room with 4 people
Only because I am one of those girls that has confidence , not to much and not to little
The next day no blood but very tired
Good location

We are still around each other all of the time
Outdoor pool was clean
Very nice hotel with super friendly staff
At the end of cooking , I add some olive oil and this makes a wonderful side dishHighly recommend
But this is very simple and delicious
It was odd
I ca nt take it
Breakfast in the restaurant was great but dinner was blah
The flavors are pure and real
Die Zimmer wurde nur gesaugt nicht einmal gewischt
  It 's my mother 's birthday , a room of 15 people bring out a cake and proceed to sing happy birthday
one night stay to attend concertdinner at winery
The oldest that lives with us has ADHDODD   the middle child has Cerebral Palsy
  in all the last 3 separations this was what he 'd always say
P

These stories kind of make a person wonder about all the times they drank and blacked out or passed out
Would love to go again
This is the same lemon juice concentrate you can find in stores and it 's really handy to have around to add to sauces , dressings , etc
Yeah right
Socially its just not accepted so their afraid to admit it
Expiration date 113
No broken neck and MRI ok Wnated A neuro consult , and awful doc was on call
He took it
Entspannend , hfliches und sehr freundliches Personal
Rooms are good   beds are comfortable
Von Idylle keine Spur
I have missed a total estimate of 4 or 5 pills
I sit here at a computer
This is my son 's 2nd favorite dish
Hi
For various things mainly for spitting , hitting , teachers and students

no period
It was a good hotel for that one night stay kind of a thing
The question What if it is my child
Only issue with the room was the dust pilled up on the shelf where the towels are kept in the bathroom
In need of desparate advice
I have the most grueso

and 6 pills light green in color and oblong in shape labeled with L2
Thanks
It works for the panang paste especially if you add a heaping tablespoon of crunchy peanut butter but the yellow , green , and red are spicy and need the full fat coconut milk to balance it out
Has anyone had similar experiences
I have been physically and emotionally abused for a very long time and to some extent sexually too
Our rooms were clean and well kept as well as all the public spaces
But there are plenty of things I do talk to myself about
Nebenhaus mit Blick auf Parkplatz
This hotel is in Lido which is a better choice than staying in crowded Venice
Martin
Den Aufenthalt vom 3 bis 852015 haben wir sehr genossen
So I guess that 's progress
Only one per
Growing up , faking everything
By comparison , Cape Cod Reduced Fat chips are less salty with at least as much potato flavor and far less greasy
Nice hotel for the price
What a treat to find a handmade American product , that competes with and wins , agai

Wir haben ein abgewohntes , schmuddeliges Appartment verordnet bekommen
Third , my new neighbors are complete jerks , not to mention a criminal gang
I was going to have to go to court
Easy to store because of the individual packaging

At the time a Doctor said it was most likely TMJ
then another a few days later , looks like an evap but its a really strong blue
But I know if this did n't happen it would have been the same thing over and over
During that time I went off a couple times , about 5 and 6yrs ago
How do I heal myself
Das Zimmer 149 liegt direkt neben einem Abluftrohr einer Klimaanlage die den ganzen Tag sehr laut lief
was just an overnite stay , was on a long trip and needed someplace to sleep for the nite
One night , it was very brief
Pool was a huge hit with our grandchildren but needs an intensive cleaning
But what else could be wrong with me
This hotel is used for Winter Romp each year
The pregnancy was extremely difficult and I was in and out of the hospital 4 times
All 

I have depression
I 'm so embarassed and humiliated that this is so obvious
I am so , so , so scared
You will look forward to going hate to leave never want to miss and crest some song friendships and a support system if those who understand
IF THE SIZE IT LESS THEN SO SHOULD THE CHARGE
Wonderful food and wine
It also has a natural sweetnessbr br The ingredients include Organic WuLi Cliff Oolong Tea 700 mg , Organic Black Tea 600 mg , Organic Green Tea Extract 100 mg , Proprietary blend 600 mg , plus some Ginseng panax , Orange Peel , Lemon Grass , and Guarana
Biscuits and gravy were typical for hotel chains
They also have a nice chewyness
It worked
i am working on having the state go over and evaluate the situation
In the beginning it was great , a typical falling in love scenario
I would stay here again
I live a normal life
Nobody cares and I just feel so sad
Continental breakfast had lots of choices and tasted great
The subtle flavors of the brine go great with cheeses
Walls extreme

...

Had a good meal in the restaurant
The hotel is located near the freeway
I just want to smile again
Fairly updated and the staff was friendly
Is there any illness whose symptom is like that
MRI comes back clean
But I 'll get to that
Breakfast discount a nice touch
I would recommend this brand of taffy   it was a delightful treat
Breakfast was very good
And it has gotten to this point where it 's been really affecting me
the pool , Hot tub and Exercise room were top shelf
  So
Symptoms Low energy , low selfesteem , difficulty making decisions , anhedoniaFall 2014May2015 , turbulent feelings journaled Fall 2013April 2015 , decrease in appetite Despite emotions , was fine until Fall 2014 could keep head afloat
wonderful acomadation
i have no idea
Can blood tests really MEASURE cortisol or are there loop holes for cortisol to be hidden in certain factors preventing an accurate assessment
Will I be okay to discontinue Rx after 9days
These are the strongest tasting salt and vinegar chips I 'v

If you are looking for a hotel right off of a freeway this one fits the bill , although way overpriced
The bed sheet had a large spot of what appeared to be an unwashable cooking grease
Makes sense
Hi
Have severe genetic Anxiety and Depression 5
If you have a problem seeing naked people enjoying the sun , walk the other side to the docks
Breakfast was definitely worth getting out of bed for
I feel bad for the employees hazardousworkingconditions
So this is the cheeper option
So we broke up
Not sure if this was an error or if the vendor intended to represent the product as Jumbo
and just walk off to go home
This was about money
However , that 's stopped now
I 'm not sure how to start or where to begin
Lots of places to swim , either at the resort or the beach
So again , Amazoncom came through with a reasonable price
You do n't need salt to hide the taste of potato chips
I have no other symptoms
It seems gross that there is so much oil on the chip that a napkin will soak it off , but it 

Could have used more than 2 bath towels
Pleasant 10 min walk along the sea front to the Water Bus
To this day none of us got an apology from mom
The room was very affordable and clean
Then the Concerta
Arrived slightly thawed
The choice is up to you
It was n't very bad in year nine but in year ten it was much worse
  but instead for its apparent success rate
I ca n't think for directions no more or anything
Very clean
He 's a completely different person
Now , the doc wants Shannon to see a psychologist
It was very light and dark brown
We were also happy that the older one will have a sibling
I excessively drink alcohol maybe once a week and smoke maybe a cigarette per day
The Munchkin fresh food feeder is an okay product , it 's kind of like sucking on food through a cloth
She refused to eat it
God What is the point in living , Just as animals do nt have souls neither wemust not have any either
I came across someone on here who mentioned music
But the thought of one day killing myself 

Can anyone give me insight to all of this
Bathrooms similar to Motels , bedsheets clean , new sheets , but bed is saggy
Honestly , my first reaction was OH NO , NOT AGAIN
I went to this site tripsitme
Great service , comfortable hotel
They have the best potato flavor
These are easily some of the most spicy chips you will ever eat , but oompletely enjoyable
Doctor at that time said lets watch
Great breakfast and wait staff
No coffee in lobby , front desk suggested I but some at the hotel restaurant
The rooms are ok , the facilities are ok , the staff is terrific
They both really go for this food
It kind of feels like I 'm dreaming , or tripping on a low dose of shrooms or something
Beautiful hotel , but very intimidating
usually eat peanut butter and crackers or a balogna sandwich with cheese
3
my husband works part time
Thanks for reading and for your time
Our marrage is on the rocks
I use a bodum tea press to prepare a pot
Another is my mood swings
but alertness had reduced
This is th

Went up for a family visit
Each room faces onto the pool with plenty of sun loungers , towels , shaded areas and comfortable seating
Really lovely hotel
Spider was in my towel
stale
But now aim hooked and have to keep a bottle on hand
My most recent purchase , however , was disappointing


**Helper function to identify voice of sentence**

In [43]:
# Identifying voice of sentence
# Helper function

# def voice_of_sentence(doc):
#     sentence_type = ""
#     for token in doc:
#         if token.dep_ == 'nsubj':
#             sentence_type = 'active'
#             break
#         elif token.dep_ == 'nsubjpass':
#             sentence_type = 'passive'

#     return sentence_type        